In [1]:
# Import the modules
import pandas as pd
import numpy as np
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
# Read in the CSV file as a Pandas DataFrame
sales_df = pd.read_csv(
    Path("Resources/US_Regional_Sales_Data.csv"),
    index_col="OrderNumber", 
    parse_dates=True, 
    infer_datetime_format=True
)

# Review the DataFrame
sales_df.head()

C:\Users\lasot\AppData\Local\Temp\ipykernel_19932\1859464474.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  sales_df = pd.read_csv(
C:\Users\lasot\AppData\Local\Temp\ipykernel_19932\1859464474.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales_df = pd.read_csv(


,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price
OrderNumber,,,,,,,,,,,,,,,
SO - 000101,In-Store,WARE-UHY1004,31/12/17,31/5/18,14/6/18,19/6/18,USD,6,15,259,12,5,0.08,"$1,001.18","$1,963.10"
SO - 000102,Online,WARE-NMK1003,31/12/17,31/5/18,22/6/18,2/7/2018,USD,14,20,196,27,3,0.08,"$3,348.66","$3,939.60"
SO - 000103,Distributor,WARE-UHY1004,31/12/17,31/5/18,21/6/18,1/7/2018,USD,21,16,213,16,1,0.05,$781.22,"$1,775.50"
SO - 000104,Wholesale,WARE-NMK1003,31/12/17,31/5/18,2/6/2018,7/6/2018,USD,28,48,107,23,8,0.08,"$1,464.69","$2,324.90"
SO - 000105,Distributor,WARE-NMK1003,10/4/2018,31/5/18,16/6/18,26/6/18,USD,22,49,111,26,8,0.10,"$1,476.14","$1,822.40"


DATA CLEANING 

In [3]:
#show data type for each column
sales_df.dtypes

Sales Channel        object
WarehouseCode        object
ProcuredDate         object
OrderDate            object
ShipDate             object
DeliveryDate         object
CurrencyCode         object
_SalesTeamID          int64
_CustomerID           int64
_StoreID              int64
_ProductID            int64
Order Quantity        int64
Discount Applied    float64
Unit Cost            object
Unit Price           object
dtype: object

In [4]:
#change columns ProcuredDate, OrderDate, ShipDate, and DeliveryDate from a format day/month/year to month/day/year

sales_df['ProcuredDate'] = pd.to_datetime(sales_df['ProcuredDate'], dayfirst=True, errors='coerce')
sales_df['OrderDate'] = pd.to_datetime(sales_df['OrderDate'], dayfirst=True, errors='coerce')
sales_df['ShipDate'] = pd.to_datetime(sales_df['ShipDate'], dayfirst=True, errors='coerce')
sales_df['DeliveryDate'] = pd.to_datetime(sales_df['DeliveryDate'], dayfirst=True, errors='coerce')


sales_df['ProcuredDate'] = pd.to_datetime(sales_df['ProcuredDate'], format='%d/%m/%Y')
sales_df['OrderDate'] = pd.to_datetime(sales_df['OrderDate'], format='%d/%m/%Y')
sales_df['ShipDate'] = pd.to_datetime(sales_df['ShipDate'], format='%d/%m/%Y')
sales_df['DeliveryDate'] = pd.to_datetime(sales_df['DeliveryDate'], format='%d/%m/%Y')



sales_df.head()

C:\Users\lasot\AppData\Local\Temp\ipykernel_19932\1789869082.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales_df['ProcuredDate'] = pd.to_datetime(sales_df['ProcuredDate'], dayfirst=True, errors='coerce')
C:\Users\lasot\AppData\Local\Temp\ipykernel_19932\1789869082.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales_df['OrderDate'] = pd.to_datetime(sales_df['OrderDate'], dayfirst=True, errors='coerce')
C:\Users\lasot\AppData\Local\Temp\ipykernel_19932\1789869082.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales_df['ShipDate'] = pd.to_datetime(sales_df[

,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price
OrderNumber,,,,,,,,,,,,,,,
SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,259,12,5,0.08,"$1,001.18","$1,963.10"
SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,196,27,3,0.08,"$3,348.66","$3,939.60"
SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,213,16,1,0.05,$781.22,"$1,775.50"
SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,107,23,8,0.08,"$1,464.69","$2,324.90"
SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,111,26,8,0.10,"$1,476.14","$1,822.40"


In [5]:
# Remove the dollar sign, commas, and extra spaces, then convert to float for 'Unit Cost'
sales_df['Unit Cost'] = sales_df['Unit Cost'].str.replace('[$,]', '', regex=True).str.strip().astype(float)

# Remove the dollar sign, commas, and extra spaces, then convert to float for 'Unit Price'
sales_df['Unit Price'] = sales_df['Unit Price'].str.replace('[$,]', '', regex=True).str.strip().astype(float)


sales_df.head()


,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price
OrderNumber,,,,,,,,,,,,,,,
SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,259,12,5,0.08,1001.18,1963.1
SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,196,27,3,0.08,3348.66,3939.6
SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,213,16,1,0.05,781.22,1775.5
SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,107,23,8,0.08,1464.69,2324.9
SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,111,26,8,0.10,1476.14,1822.4


In [6]:
#add column "delivery_time" to the dataframe which is the difference between the order date and the delivery date
sales_df["delivery_time_days"] = sales_df["DeliveryDate"] - sales_df["OrderDate"]
#remove "days" from the column and convert to integer
sales_df["delivery_time_days"] = sales_df["delivery_time_days"].astype(str).str.replace('days', '').astype(int)
sales_df.head()

,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price,delivery_time_days
OrderNumber,,,,,,,,,,,,,,,,
SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,259,12,5,0.08,1001.18,1963.1,19
SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,196,27,3,0.08,3348.66,3939.6,32
SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,213,16,1,0.05,781.22,1775.5,31
SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,107,23,8,0.08,1464.69,2324.9,7
SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,111,26,8,0.10,1476.14,1822.4,26


In [7]:
#create a new column "price_before_disc" which is the price of the item before the discount is applied
sales_df["price_before_disc"] = sales_df["Unit Price"] / (1 - sales_df["Discount Applied"])
sales_df.head()

,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price,delivery_time_days,price_before_disc
OrderNumber,,,,,,,,,,,,,,,,,
SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,259,12,5,0.08,1001.18,1963.1,19,2133.804348
SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,196,27,3,0.08,3348.66,3939.6,32,4282.173913
SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,213,16,1,0.05,781.22,1775.5,31,1868.947368
SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,107,23,8,0.08,1464.69,2324.9,7,2527.065217
SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,111,26,8,0.10,1476.14,1822.4,26,2024.888889


In [8]:
#show basic statistics for the dataframe
sales_df.describe()

,ProcuredDate,OrderDate,ShipDate,DeliveryDate,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price,delivery_time_days,price_before_disc
count,7991,7991,7991,7991,7991.000000,7991.000000,7991.000000,7991.000000,7991.000000,7991.000000,7991.000000,7991.000000,7991.000000,7991.000000
mean,2019-05-29 05:11:01.794518784,2019-09-15 11:01:09.828557312,2019-09-30 15:04:26.249530624,2019-10-06 03:10:06.832686592,14.384307,25.457014,183.850081,23.771743,4.525341,0.115649,1431.911513,2284.536504,20.672882,2617.656999
min,2017-12-31 00:00:00,2018-05-31 00:00:00,2018-06-02 00:00:00,2018-06-07 00:00:00,1.000000,1.000000,1.000000,1.000000,1.000000,0.050000,68.680000,167.500000,3.000000,176.315789
25%,2018-10-27 00:00:00,2019-01-16 12:00:00,2019-01-31 00:00:00,2019-02-06 00:00:00,8.000000,13.000000,91.000000,12.000000,3.000000,0.050000,606.120000,1031.800000,14.000000,1136.086957
50%,2019-05-15 00:00:00,2019-09-15 00:00:00,2019-09-30 00:00:00,2019-10-05 00:00:00,14.000000,25.000000,183.000000,24.000000,5.000000,0.080000,1080.580000,1849.200000,21.000000,2062.111111
75%,2020-03-10 00:00:00,2020-05-12 00:00:00,2020-05-28 00:00:00,2020-06-01 00:00:00,21.000000,38.000000,276.000000,36.000000,7.000000,0.150000,2040.250000,3611.300000,27.000000,4041.157895
max,2020-09-26 00:00:00,2020-12-30 00:00:00,2021-01-24 00:00:00,2021-02-02 00:00:00,28.000000,50.000000,367.000000,47.000000,8.000000,0.400000,5498.560000,6566.000000,38.000000,10854.000000
std,NaN,NaN,NaN,NaN,7.986086,14.414883,105.903946,13.526545,2.312631,0.085018,1112.413063,1673.096364,8.295398,1965.483162


In [9]:

# Assign "Early delivery" for <15 days and "Regular delivery" for >=15 days
sales_df["delivery_time_bucket"] = np.where(sales_df["delivery_time_days"] < 15, 
                                           "Early delivery", 
                                           "Regular delivery")
sales_df.head()

,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price,delivery_time_days,price_before_disc,delivery_time_bucket
OrderNumber,,,,,,,,,,,,,,,,,,
SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,259,12,5,0.08,1001.18,1963.1,19,2133.804348,Regular delivery
SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,196,27,3,0.08,3348.66,3939.6,32,4282.173913,Regular delivery
SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,213,16,1,0.05,781.22,1775.5,31,1868.947368,Regular delivery
SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,107,23,8,0.08,1464.69,2324.9,7,2527.065217,Early delivery
SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,111,26,8,0.10,1476.14,1822.4,26,2024.888889,Regular delivery


In [10]:
#show the number of orders in each delivery time bucket
sales_df["delivery_time_bucket"].value_counts()

delivery_time_bucket
Regular delivery    5842
Early delivery      2149
Name: count, dtype: int64

IMPACT OF VARIABLES ON DELIVERY TIMES and delivery time prediction - based on sales channel, sales team, and warehouse 

In [11]:
# encoding categorical variables 
#replace "Sales Channel" categories with integers - "Distributor"=0, "Online"=1, "In-Store"=2, "Wholesale"=3
sales_df["Sales Channel"] = sales_df["Sales Channel"].replace({"Distributor": 0, "Online": 1, "In-Store": 2, "Wholesale": 3})
#replace WarehouseCode  categories with integers - "WARE-MKL1006"=6, "WARE-NBV1002"=2, "WARE-NMK1003"=3, "WARE-PUJ1005"=5, "WARE-UHY1004"=4, "WARE-XYS1001"=1
sales_df["WarehouseCode"] = sales_df["WarehouseCode"].replace({"WARE-MKL1006": 6, "WARE-NBV1002": 2, "WARE-NMK1003": 3, "WARE-PUJ1005": 5, "WARE-UHY1004": 4, "WARE-XYS1001": 1})
sales_df.head()

,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price,delivery_time_days,price_before_disc,delivery_time_bucket
OrderNumber,,,,,,,,,,,,,,,,,,
SO - 000101,2,4,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,259,12,5,0.08,1001.18,1963.1,19,2133.804348,Regular delivery
SO - 000102,1,3,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,196,27,3,0.08,3348.66,3939.6,32,4282.173913,Regular delivery
SO - 000103,0,4,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,213,16,1,0.05,781.22,1775.5,31,1868.947368,Regular delivery
SO - 000104,3,3,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,107,23,8,0.08,1464.69,2324.9,7,2527.065217,Early delivery
SO - 000105,0,3,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,111,26,8,0.10,1476.14,1822.4,26,2024.888889,Regular delivery


In [12]:
#change name of sales_df to sales_df_encoded
sales_df_encoded = sales_df

In [13]:
#drop columns that are not needed for the analysis
sales_df_encoded = sales_df_encoded.drop(columns=['ProcuredDate', 'OrderDate', 'ShipDate', 'DeliveryDate', 'CurrencyCode','delivery_time_days'])

In [14]:
from sklearn.model_selection import train_test_split

#Split the data into training and testing sets based on delivery time bucket

X = sales_df_encoded.drop('delivery_time_bucket', axis=1)
y = sales_df_encoded['delivery_time_bucket']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
from sklearn.ensemble import RandomForestClassifier

#Train the random forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [16]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7235772357723578
                  precision    recall  f1-score   support

  Early delivery       0.32      0.02      0.03       433
Regular delivery       0.73      0.99      0.84      1166

        accuracy                           0.72      1599
       macro avg       0.52      0.50      0.44      1599
    weighted avg       0.62      0.72      0.62      1599



In [17]:
feature_importances = clf.feature_importances_

# Create a DataFrame for visualization
importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Sort by importance
importances_df = importances_df.sort_values(by='Importance', ascending=False)

print(importances_df)

              Feature  Importance
8           Unit Cost    0.137206
4            _StoreID    0.131887
10  price_before_disc    0.127142
9          Unit Price    0.116443
5          _ProductID    0.112411
3         _CustomerID    0.111649
2        _SalesTeamID    0.090974
6      Order Quantity    0.062722
7    Discount Applied    0.050654
1       WarehouseCode    0.032836
0       Sales Channel    0.026076


LOGISTIC REGRESSION - DELIVERY TIME PREDICTION MODEL

In [18]:
# show number of rows for each category in loan_status
sales_df_encoded["delivery_time_bucket"].value_counts()

delivery_time_bucket
Regular delivery    5842
Early delivery      2149
Name: count, dtype: int64

In [19]:
# Separate the data into labels and features
# Separate the y variable, the labels
y = sales_df_encoded["delivery_time_bucket"]

# Separate the X variable, the features
X = sales_df_encoded.drop(columns=["delivery_time_bucket"])

In [20]:
# Review the y variable Series
print(f"Labels: {y[:10]}")

Labels: OrderNumber
SO - 000101    Regular delivery
SO - 000102    Regular delivery
SO - 000103    Regular delivery
SO - 000104      Early delivery
SO - 000105    Regular delivery
SO - 000106      Early delivery
SO - 000107      Early delivery
SO - 000108    Regular delivery
SO - 000109    Regular delivery
SO - 000110    Regular delivery
Name: delivery_time_bucket, dtype: object


In [21]:
# Review the X variable DataFrame
print(f"Data: {X[:10]}")

Data:              Sales Channel  WarehouseCode  _SalesTeamID  _CustomerID  \
OrderNumber                                                            
SO - 000101              2              4             6           15   
SO - 000102              1              3            14           20   
SO - 000103              0              4            21           16   
SO - 000104              3              3            28           48   
SO - 000105              0              3            22           49   
SO - 000106              1              5            12           21   
SO - 000107              2              1            10           14   
SO - 000108              2              5             6            9   
SO - 000109              2              5             4            9   
SO - 000110              2              4            10           33   

             _StoreID  _ProductID  Order Quantity  Discount Applied  \
OrderNumber                                               

In [22]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(5993, 11)

In [23]:
X_test.shape

(1998, 11)

In [24]:
from sklearn.preprocessing import StandardScaler

# Assuming X_train is your training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [25]:
#Create a logistic regression model
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(max_iter=10000)


# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

In [26]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
OrderNumber,,
SO - 0005035,Regular delivery,Early delivery
SO - 0004929,Regular delivery,Regular delivery
SO - 0007226,Regular delivery,Regular delivery
SO - 0004615,Regular delivery,Regular delivery
SO - 0004304,Regular delivery,Early delivery
...,...,...
SO - 0002866,Regular delivery,Regular delivery
SO - 0002542,Regular delivery,Early delivery
SO - 0005876,Regular delivery,Early delivery


In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions)

array([[   0,  537],
       [   0, 1461]], dtype=int64)

In [28]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Early delivery","Regular delivery"], columns=["Early delivery","Regular delivery"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Display results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


Confusion Matrix


,Early delivery,Regular delivery
Early delivery,0,537
Regular delivery,0,1461


Accuracy Score : 0.7312312312312312
Classification Report
                  precision    recall  f1-score   support

  Early delivery       0.00      0.00      0.00       537
Regular delivery       0.73      1.00      0.84      1461

        accuracy                           0.73      1998
       macro avg       0.37      0.50      0.42      1998
    weighted avg       0.53      0.73      0.62      1998



c:\Users\lasot\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lasot\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lasot\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
